In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.3]  # nem tul kis spread

for col in df.columns:
    if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
        df.drop(col, axis=1, inplace=True)
    # if 'nr_trades' in col:
    #     df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)
    pass

df_min = df[df.wave_direction == 'min'].copy()
df_max = df[df.wave_direction == 'max'].copy()

df_min.drop('wave_direction', axis=1, inplace=True)
df_max.drop('wave_direction', axis=1, inplace=True)
df = None
print(f'min dataset: {len(df_min)}')
print(f'max dataset: {len(df_max)}')

df_min.sort_index(axis=1, inplace=True)
df_max.sort_index(axis=1, inplace=True)

display(df_max)

# df_max.last_price_delta_since_stabilized = df_max.last_price_delta_since_stabilized * -1
df_max.last_price_delta_since_stabilized.describe()

total dataset: 181297 rows
min dataset: 61672
max dataset: 62725


,0_amount_mean,1_amount_mean,2_amount_mean,3_amount_mean,4_amount_mean,last_price_delta_since_stabilized,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_spread
index,,,,,,,,,,
0,0.0305,0.0815,0.0792,0.0767,0.0725,-0.78,0.0706,108,-0.4798,0.91
0,0.0078,0.0072,0.0122,0.0173,0.0301,-1.84,0.0295,186,0.2329,1.67
0,0.0348,0.0531,0.0503,0.0489,0.0481,-0.74,0.0469,151,-0.8172,1.83
0,0.0031,0.0027,0.0036,0.0058,0.0188,-0.89,0.0167,170,-0.9874,1.81
0,0.0318,0.0315,0.0317,0.0341,0.0360,0.00,0.0345,822,-0.5774,3.55
...,...,...,...,...,...,...,...,...,...,...
0,0.0591,0.0402,0.0305,0.0462,0.0412,-2.92,0.0370,162,-1.3984,2.10
0,0.0174,0.0065,0.0060,0.0056,0.0054,-2.23,0.0120,148,-0.9416,1.34
0,0.0135,0.0492,0.0470,0.0447,0.0426,-1.80,0.0416,74,-1.9380,2.24


count    62725.000000
mean        -0.578860
std          0.993386
min        -17.160000
25%         -0.730000
50%         -0.200000
75%          0.000000
max          0.000000
Name: last_price_delta_since_stabilized, dtype: float64

In [2]:
a = df_min.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))

a = df_max.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))


last_price_delta_since_stabilized    1000.000000
stabilized_spread                     127.382395
stabilized_amount_mean                 66.918363
4_amount_mean                          57.699117
3_amount_mean                          55.419810
2_amount_mean                          51.350720
1_amount_mean                          50.199775
0_amount_mean                          48.921019
stabilized_gasp                        15.069423
stabilized_at_ms                     -299.774501
Name: last_price_delta_since_stabilized, dtype: float64

last_price_delta_since_stabilized    1000.000000
stabilized_at_ms                      294.647192
stabilized_gasp                         5.988095
1_amount_mean                         -41.875984
2_amount_mean                         -41.880710
0_amount_mean                         -42.161516
3_amount_mean                         -48.790441
4_amount_mean                         -52.477130
stabilized_amount_mean                -64.315900
stabilized_spread                    -124.628688
Name: last_price_delta_since_stabilized, dtype: float64

In [3]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler


def train(df, loss_function):
    # model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function=loss_function, random_state=0, verbose=False, iterations=1500)
    model = CatBoostRegressor(learning_rate=0.15, depth=6, loss_function=loss_function, random_state=0, verbose=False)
    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    df = df.copy()
    y = df['last_price_delta_since_stabilized']
    X = df.drop('last_price_delta_since_stabilized', axis=1)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_valid, y_valid)
    display(score)
    return model, score


# min_model, min_score = train(df_min, loss_function='MAPE')
# max_model, max_score = train(df_max, loss_function='MAPE')

# min_model, min_score = train(df_min, loss_function='RMSE')
# max_model, max_score = train(df_max, loss_function='RMSE')

# min_model, min_score = train(df_min, loss_function='Poisson') # 0.12
# max_model, max_score = train(df_max, loss_function='Poisson') # 0.14

# min_model, min_score = train(df_min, loss_function='Huber:delta=0.4')
# max_model, max_score = train(df_max, loss_function='Huber:delta=0.4')

# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.8')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.6')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.5')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.4')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.3')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.2')

max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')



2023-02-08 21:15:20.885 | INFO     | spreadsurfer.balance:<module>:9 - balance config: {'panic_below_total': 390, 'panic_countdown_from': 8, 'panic_below_profitability': -0.00015}
2023-02-08 21:15:20.886 | INFO     | spreadsurfer.waves:<module>:15 - wave detection config: {'min_length': 6, 'investigate_length': 5, 'stabilized_threshold': 0.01, 'max_delta_ms_to_create_order': 500, 'collect_last_n_frames': 6, 'collect_last_n_wave_prices': 10}
2023-02-08 21:15:20.930 | INFO     | spreadsurfer.orders:<module>:18 - order config: {'orders_disabled': False, 'test_mode': True, 'max_nr_orders_limited': True, 'max_nr_orders_created': 20, 'base_amount': 0.0015, 'skip_order_on_spread_below': 0.2, 'skip_order_on_spread_above': 10, 'low_spread_limit': 0.2, 'recv_window': 100, 'cancel_far_order_after_ms': 400}


-0.16813498332933108

In [14]:
from spreadsurfer import now_isoformat

save = True
if save:
    # min_model.save_model(f'./models/{now_isoformat()}-min_-quantile0.7-score-{1000 * round(min_score, 3)}.cat')
    max_model.save_model(f'./models/{now_isoformat()}-max_quantile0.7-score-{1000 * round(max_score, 3)}.cat')
